<a href="https://colab.research.google.com/github/RodolfoFerro/satelitesyneuronas/blob/main/notebooks/Introducci%C3%B3n_a_la_API_de_Earth_Engine_Python_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Una introducción a la API de Python de Earth Engine

Autor: Ana Patricia Ruiz Beltrán
Texto introductorio por: guiattard


Dentro de la última década, una gran cantidad de datos geoespaciales, como datos satelitales (p. ej., temperatura de la superficie terrestre, vegetación) o la salida de modelos a gran escala, incluso globales (p. ej., velocidad del viento, recarga de aguas subterráneas), han estado disponibles gratuitamente de múltiples agencias nacionales y universidades (p. ej., NASA, USGS, NOAA y ESA).

Estos datos geoespaciales son utilizados diariamente por científicos e ingenieros de todos los campos para predecir el clima, prevenir desastres, asegurar el suministro de agua o estudiar las consecuencias del cambio climático. Al utilizar estos datos geoespaciales, surgen algunas preguntas:

¿Qué datos están disponibles y dónde se pueden encontrar?

¿Cómo podemos acceder a estos datos?

¿Cómo podemos manipular estos petabytes de datos?

En este tutorial, se presenta una introducción a la API de Python de Google Earth Engine: https://developers.google.com/earth-engine/guides/python_install. Después de algunas configuraciones y una exploración del Catálogo de datos de Earth Engine, veremos cómo manejar conjuntos de datos geoespaciales con pandas: https://pandas.pydata.org/ y crear algunos gráficos con matplotlib.

Primero, veremos cómo obtener las series temporales de una variable para una región de interés. Se aplicará este procedimiento para conocer el comportamiento de la vegetación verde sobre el tiempo. En segundo lugar, detallaremos los procedimientos para la creación de mapas estáticos y la exportación de resultados como GeoTIFF.

Finalmente, se introducirá la biblioteca folium para crear mapas interactivos. En esta última parte, veremos cómo incluir algunos conjuntos de datos de GEE como capas de mosaico de un mapa de folium.

## Corre esto primero

Primero, ejecuta la siguiente celda para inicializar la API. La salida contendrá instrucciones sobre cómo otorgar acceso a Earth Engine a este notebook usando tu cuenta.

In [ ]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='rodo-ferro')

## Empezemos con las "Collections"

En el Catálogo de Datos de Earth Engine, los conjuntos de datos pueden ser de diferentes tipos:

* Features (Características) que son objetos geométricos con una lista de propiedades. Por ejemplo, una cuenca hidrográfica con algunas propiedades como nombre y área, es un ee.Feature.
* Images (Imágenes) que son como las características, pero pueden incluir varias bandas. Por ejemplo, la elevación del terreno proporcionada por el USGS aquí: https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003 es una ee.Image.
* Collections (Colecciones) que son grupos de características o imágenes. Por ejemplo, las Capas de Unidades Administrativas Globales (Global Administrative Unit Layers) que proporcionan los límites administrativos son una ee.* FeatureCollection: https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0 y el conjunto de datos de Temperatura de la Superficie Terrestre (Land Surface Temperature) de MODIS es una ee.ImageCollection: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1.


Si deseas obtener más información sobre los diferentes modelos de datos, puede visitar la Guía del Usuario de Earth Engine: https://developers.google.com/earth-engine.

Explora los datos que hay de imágenes satelitales en Google Earth Engine aquí: https://developers.google.com/earth-engine/datasets

Ahora, antes de importar la colección de imágenes satelitales, definamos nuestra área de estudio. Ahorita está la de Aguascalientes pero sé libre de modificar las coordenadas.

In [ ]:
# Define la geometría usando tus coordenadas de interés
geometry = ee.Geometry.Polygon([
    [
        [-102.52664708407724, 21.730974562304468],
        [-102.12152623446786, 21.730974562304468],
        [-102.12152623446786, 22.105537299784615],
        [-102.52664708407724, 22.105537299784615],
        [-102.52664708407724, 21.730974562304468]
    ]
])

In [ ]:
# Crear un mapa centrado en la zona de interés

Map = geemap.Map(center=[21.92, -102.32], zoom=10)
Map.addLayer(geometry, {}, 'Zona de interés')
Map

# Interactúa con todas las opciones que tienes en la ventana.
#Pruébalo tú: Haz otro código similar, pero juega con las variables de entrada

In [ ]:
# Ahora sí, importemos primero la colección de imágenes armonizadas de Sentinel-2

# Selección de imágenes Sentinel-2
S2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2017-04-01', '2025-07-01')
    .filterBounds(geometry)
)


### Cálculo del Índice NDVI

Con esta colección de imágenes, podemos calcular fácilmente el Valor del Índice de Vegetación de Diferencia Normalizada (NDVI) para cada una de ellas. El NDVI es un indicador clave de la salud y densidad de la vegetación, y se deriva de una ecuación que utiliza la reflectancia de la Banda Infrarroja Cercana (NIR) y la Banda Roja (RED) de una imagen satelital.

La fórmula del NDVI se expresa de la siguiente manera:

$$
NDVI = \frac{NIR - RED}{NIR + RED}
$$

Donde:

- **NIR** representa la reflectancia en la Banda Infrarroja Cercana (en tu caso, la Banda 8).
- **RED** representa la reflectancia en la Banda Roja (en tu caso, la Banda 4).

Este cálculo nos permitirá generar un nuevo conjunto de datos que refleje la distribución y vitalidad de la vegetación en las áreas de estudio.

In [ ]:
#Cálculo del NDVI

def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

S2 = S2.map(addNDVI)

In [ ]:
#Aquí pondremos la serie de tiempo, de todo el polígono de interés, para ver cómo la vegetación verde se ha comportado sobre el tiempo

#Aquí importamos la paquetería que necesitamos para hacer el plot
import pandas as pd
import matplotlib.pyplot as plt


#Hacemos una función para obtener sólamente el NDVI
def get_ndvi_time_series(collection, geometry):
    def extract(img):
        stats = img.select('NDVI').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=10,
            maxPixels=1e9
        )
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, 'NDVI': stats.get('NDVI')})
    series = collection.map(extract).filter(ee.Filter.notNull(['NDVI']))
    data = series.getInfo()['features']
    dates = [f['properties']['date'] for f in data]
    ndvis = [f['properties']['NDVI'] for f in data]
    return pd.DataFrame({'date': dates, 'NDVI': ndvis})

#Extraemos los datos y los acomodamos por fecha

ndvi_df = get_ndvi_time_series(S2, geometry)
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'])
ndvi_df = ndvi_df.sort_values('date')

#Estos son los detalles para adornar la  gráfica
plt.figure(figsize=(10,5))
plt.plot(ndvi_df['date'], ndvi_df['NDVI'])
plt.title('Serie temporal de NDVI')
plt.xlabel('Fecha')
plt.ylabel('NDVI')
plt.show()

In [ ]:
import plotly.express as px

fig = px.line(ndvi_df, x='date', y='NDVI', title='Serie temporal de NDVI')
fig.show()

Si vemos esta gráfica podemos observar que hay algo que no cuadra, ¿Puedes identificar qué es? Observa los valores de NDVI... 0.0 corresponde a no vegetación y 1 es el valor de saturación. ¿Puede de un día a otro desaparecer la vegetación? ¿Qué podría estar causando esto?

## Enmascarado de nubes y filtrado avanzado

Para poder encontrar imágenes que puedan ser útiles para nuestra visualización de datos y/o análisis es necesario hacer un filtro de aquellas imágenes que tienen nubosidad. Para eso es es que el siguiente código hace un filtro de aquellas imágenes que tienen una probabilidad alta de nubosidad.


In [ ]:
def maskClouds(image):
    cloud_prob = image.select('MSK_CLDPRB')
    mask = cloud_prob.lt(20)
    return image.updateMask(mask).copyProperties(image, ['system:time_start'])

def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def calculateOverlapPercentage(image):
    imageGeometry = image.geometry()
    intersection = imageGeometry.intersection(geometry, 0.001)
    intersectionArea = intersection.area()
    imageArea = imageGeometry.area()
    overlapPercentage = intersectionArea.divide(imageArea).multiply(100)
    return image.set('overlap_percentage', overlapPercentage)

S2_filtered = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2018-12-14', '2025-04-01')
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
    .map(maskClouds)
    .map(addNDVI)
    .map(calculateOverlapPercentage)
    .filter(ee.Filter.gte('overlap_percentage', 10))
    .map(lambda img: img.clip(geometry))
)

In [ ]:
#Repite la serie de tiempo usando S2_filtered en vez de S2 para graficar solo imágenes con nubes filtradas.


def get_ndvi_time_series(collection, geometry):
    def extract(img):
        stats = img.select('NDVI').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=10,
            maxPixels=1e9
        )
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, 'NDVI': stats.get('NDVI')})
    series = collection.map(extract).filter(ee.Filter.notNull(['NDVI']))
    data = series.getInfo()['features']
    dates = [f['properties']['date'] for f in data]
    ndvis = [f['properties']['NDVI'] for f in data]
    return pd.DataFrame({'date': dates, 'NDVI': ndvis})

ndvi_df = get_ndvi_time_series(S2_filtered, geometry)
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'])
ndvi_df = ndvi_df.sort_values('date')

plt.figure(figsize=(10,5))
plt.plot(ndvi_df['date'], ndvi_df['NDVI'])
plt.title('Serie temporal de NDVI')
plt.xlabel('Fecha')
plt.ylabel('NDVI')
plt.show()

##Otros índices de Vegetación

Investiga otros índices de vegetación que puedan ser estimados con Sentinel-2.
Calcúlalo abajo y elabora la gráfica.

## Visualizando los datos en 2D

Ahora, visualizaremos las imágenes de nuestra colección en 2D. Tenemos muchas imágenes, así que seleccionaremos sólo 2, y que sean contrastantes. Una que sea de época de secas y otra de época de lluvias. Podemos usar el índice de vegetación del NDVI como un proxi para encontrar esto. El máximo y el mínimo.

Recapitulando, ¿cuántas imágenes tenemos en nuestra colección?

In [ ]:
print('Número de imágenes:', S2_filtered.size().getInfo())

Ahora calcula el número de imágenes que tenías, antes de que aplicáramos el filtro de nubes a las imágenes de la colección de Sentinel-2

In [ ]:
#Ejercicio: Calcula el número de imágenes antes del filtro de nubes para nuestra colección S2

Para poder hacer esto, primero tenemos que calcular el NDVI promedio de toda nuestra área de estudio por imágen. Después, seleccionamos la imagen con los valores mínimos y los valores máximos de NDVI.

In [ ]:
##Creación de una función para calcular el NDVI por imágen

def addNDVImean(img):
    mean = img.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(), #obtener el promedio del NDVI
        geometry=geometry, #área de interés definida anteriormente en el código
        scale=10, #resolución espacial, en este caso es Sentinel-2, así que es de 10 metris
        maxPixels=1e9 #número máximo de pixeles para procesar
    ).get('NDVI')
    return img.set('NDVI_mean', mean)
##Aplicación de la función en mi colección de datos

ndviStats = S2_filtered.map(addNDVImean)

#Acomodar las imágenes por sus valores de NDVI y seleccionar la primera y la última
minNDVIImage = ndviStats.sort('NDVI_mean', True).first()
maxNDVIImage = ndviStats.sort('NDVI_mean', False).first()

¿A qué fecha corresponden estas imágenes? Utilizaremos un módulo para encontrar las fechas de las imágenes que seleccionamos

In [ ]:
import datetime  # Este es un módulo para convertir fechas, ya que las que obtenemos
#están en milisegundos

# Obtenemos el tiempo de la imagen, que está en milisegundos desde el 1 de enero de 1970
timestamp = minNDVIImage.get('system:time_start').getInfo()

# Convertimos la marca de tiempo a una fecha legible utilizando el móddulo
date = datetime.datetime.utcfromtimestamp(timestamp / 1000)
print("Fecha de la imagen con NDVI mínimo:", date)

In [ ]:
#Ahora haz lo mismo con la fecha del NDVI máximo

##Visualizando el mapa

Aquí visualizaremos las imágenes que queremos en el mapa.

In [ ]:
# Creación de la variable mapa, en donde pondremos primero las coordenadas de nuestro centro de interés,
# y después definiremos el acercamiento con zoom
Map = geemap.Map(center=[22.25, -102.25], zoom=10)
#Agregamos cada una de las imágenes que queremos. Estas son las imágenes que definimos anteriormente. Agregamos los valores mínimos y máximos.
#Puedes cambiar este parámetro para tu valor máximo de NDVI, para tener un mayor contraste. Después se define la paleta de colores
#y finalmente ponemos el nombre de la variable a visualizar en nuestro mapa
Map.addLayer(minNDVIImage.select('NDVI'), {'min': 0, 'max': 0.8, 'palette': ['brown', 'yellow', 'green']}, 'NDVI secas')
Map.addLayer(maxNDVIImage.select('NDVI'), {'min': 0, 'max': 0.8, 'palette': ['brown', 'yellow', 'green']}, 'NDVI lluvias')
Map.addLayer(minNDVIImage, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel 2 - Color Verdadero - secas')
Map.addLayer(maxNDVIImage, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel 2 - Color Verdadero - lluvias')
#Aquí llamamos al mapa para que se imprima
Map

Así como aplicas filtros en instagram, utilizamos las mismas técnicas para poder generar un mejor contraste en las imágenes. Para esto, podemos utilizar, por ejemplo, la desviación estándar.

¿Qué es la desviación estándar?

Veamos los histogramas de las bandas de interés, para encontrar los valores de los pixeles, y con esto poder entender cómo es que funcionan los contrastes al asignar los máximos y mínimos en la visualización de imágenes.

In [ ]:
#Selección de las bandas de interés

bands = ['B2', 'B3', 'B4', 'B8']

#Aplicar un "loop" o "for" para hacer muchas gráficas al mismo tiempo
for band in bands:
    values = minNDVIImage.select(band).reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=geometry,
        scale=10,
        maxPixels=1e8
    ).get(band).getInfo()

# Graficamos
    plt.figure(figsize=(8, 4)) #Tamaño del plot
    plt.hist(values, bins=50, edgecolor='black') #Intervalos de agrupación de los valores
    plt.title(f'Histograma de la banda espectral {band}') #Aquí se escribe automáticamente el nombre de la banda
    plt.xlabel('Valor del píxel')
    plt.ylabel('Frecuencia')
    plt.grid(True)
    plt.show()

##Haz tu propio mapa

Ahora, haz tu propio mapa jugando con distintos valores de bandas, pero ya no en **color verdadero** (*True Colour*), sino con **falso color**, y ajusta los valores mínimos y máximos con la desviación estándar.

[Aquí](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/composites/) algunos ejemplos de falso color para utilizar diferentes combinaciones de bandas y resaltar distintos atributos de la superficie terrestre.

## Exportando un archivo GeoTIFF

Después de manipular conjuntos de datos en Earth Engine, puede que necesites exportar una ee.Image resultante a un archivo GeoTIFF. Por ejemplo, para usarla como entrada en un modelo numérico fuera de Earth Engine, o para superponerla con archivos georreferenciados personales en tu SIG favorito. Hay múltiples formas de hacerlo (consulta la sección de [Exporting](https://developers.google.com/earth-engine/guides/exporting#exporting-images) en la Guía de Desarrolladores).

Aquí exploramos dos opciones:

* Guardar la ee.Image deseada en Google Drive.
* Descargar directamente la imagen.



## Guarda un archivo GeoTIFF en tu Google Drive

Para exportar la ee.Image a Google Drive, debemos definir una tarea y comenzarla. Tenemos que especificar el tamaño de los píxeles (aquí 10 m), la proyección (aquí EPSG:4326), el formato del archivo (aquí GeoTIFF), la región de interés (aquí el área de Aguascalientes definida anteriormente), y el archivo se exportará al directorio principal de Google Drive y se nombrará según el fileNamePrefix que elijamos.

In [ ]:
#Llamamos a la imágen para conocer sus características

minNDVIImage

In [ ]:
#Primero seleccionamos qué bandas queremos exportar

bands_to_select = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B11', 'B12']

In [ ]:
task = ee.batch.Export.image.toDrive(image=sub_image,
                                     description='Aguascalientes_Sentinel2_lluvias',
                                     scale=10,
                                     region=geometry,
                                     fileNamePrefix='Ags_S2',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

Luego podemos verificar el estado de nuestra tarea (nota: la tarea también se registrará en la lista de tareas del Editor de código JavaScript [Code Editor's](https://code.earthengine.google.com/)) usando el método status(). Dependiendo del tamaño de la solicitud, podríamos ejecutar esta celda varias veces hasta que el estado de la tarea cambie a 'COMPLETED' (en orden, el estado de la tarea de exportación es 'READY', luego 'RUNNING' y finalmente 'COMPLETED').

In [ ]:
task.status()

## Mapa interactivo usando folium

Para mostrar estos conjuntos de datos de GEE en un mapa interactivo, permítame presentarles [Folium](https://python-visualization.github.io/folium/). Folium es una librería de Python basada en [leaflet.js](https://leafletjs.com/) (una librería de JavaScript de código abierto para mapas interactivos adaptados a dispositivos móviles) que puedes usar para crear mapas interactivos. Folium es compatible con capas WMS, GeoJSON, capas vectoriales y capas de mosaico, lo que lo hace muy conveniente y sencillo para visualizar los datos que manipulamos con Python. Creamos nuestro primer mapa interactivo con una sola línea de código, especificando la ubicación donde queremos centrar el mapa, el nivel de zoom y las dimensiones principales del mapa:


In [ ]:
import folium

# Define el centro y zoom
center = [22.25, -102.25]
zoom = 10

# Crea el mapa base con folium
m = folium.Map(location=center, zoom_start=zoom)

# Define tus parámetros de visualización
ndvi_vis = {'min': 0, 'max': 0.8, 'palette': ['brown', 'yellow', 'green']}
rgb_secas_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
rgb_lluvias_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 2225, 'max': 4045}

# Obtén el mapa base URL para cada imagen y agrégala a folium
def add_ee_layer(folium_map, ee_image, vis_params, name):
    map_id_dict = ee.Image(ee_image).getMapId(vis_params)
    tile_url = map_id_dict['tile_fetcher'].url_format

    folium.TileLayer(
        tiles=tile_url,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(folium_map)

# Agrega capas al mapa
add_ee_layer(m, minNDVIImage.select('NDVI'), ndvi_vis, 'NDVI secas')
add_ee_layer(m, maxNDVIImage.select('NDVI'), ndvi_vis, 'NDVI lluvias')
add_ee_layer(m, minNDVIImage, rgb_secas_vis, 'True Color - NDVI secas')
add_ee_layer(m, maxNDVIImage, rgb_lluvias_vis, 'True Color - NDVI lluvias')

# Agrega control de capas
folium.LayerControl().add_to(m)

# Muestra el mapa
m

Finalmente, el mapa se puede guardar en formato *HTML* usando el método save() de Folium, especificando el nombre del archivo como un argumento de este método. Si ejecutas esta celda usando Google Colab, tu archivo *HTML* se guardará en la carpeta content de tu entorno Colab. Si ejecutas esta celda localmente, el archivo se guardará en tu directorio de trabajo actual. Luego, podrás abrir tu archivo *HTML* con tu navegador favorito.


In [ ]:
m.save('mapa.html')

##Documentación

La documentación completa de la API de Python de Google Earth Engine está disponible [aquí](https://developers.google.com/earth-engine/api_docs)

La Guía del Usuario de Google Earth Engine está disponible [aquí](https://developers.google.com/earth-engine)

Algunos tutoriales están disponibles [aquí](https://developers.google.com/earth-engine/tutorials)